In [1]:
# Load the Drive helper and mount
from google.colab import drive

# This will prompt for authorization.
drive.mount('/content/drive', force_remount=True)

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [2]:
!ls

drive  sample_data


In [3]:
cd /content/drive/My Drive

/content/drive/My Drive


In [4]:
!pwd

/content/drive/My Drive


In [5]:
!ls

autoencoder8.h5			  phaseThree_joint_TPU.ipynb
embeddings_all_list.npy		  phaseThree_joint_tt.ipynb
hdf5				  phaseTwo_generateEmbeddings.ipynb
mobile_regress_5.h5		  phaseTwo_ImageNet.ipynb
phaseOne_Autoencoder.ipynb	  phaseTwo_ImageNet_mobile_6final.ipynb
phaseThree_joint_model_GPU.ipynb


In [0]:
!pip install -q keras

In [7]:
import tensorflow as tf
tf.test.gpu_device_name()

'/device:GPU:0'

In [8]:
from tensorflow.python.client import device_lib
device_lib.list_local_devices()

[name: "/device:CPU:0"
 device_type: "CPU"
 memory_limit: 268435456
 locality {
 }
 incarnation: 3281039656796261463, name: "/device:XLA_CPU:0"
 device_type: "XLA_CPU"
 memory_limit: 17179869184
 locality {
 }
 incarnation: 6350746327791210547
 physical_device_desc: "device: XLA_CPU device", name: "/device:XLA_GPU:0"
 device_type: "XLA_GPU"
 memory_limit: 17179869184
 locality {
 }
 incarnation: 8080055521075783952
 physical_device_desc: "device: XLA_GPU device", name: "/device:GPU:0"
 device_type: "GPU"
 memory_limit: 11281553818
 locality {
   bus_id: 1
   links {
   }
 }
 incarnation: 9913140652098982381
 physical_device_desc: "device: 0, name: Tesla K80, pci bus id: 0000:00:04.0, compute capability: 3.7"]

# **Joint model**

In [0]:
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D

In [10]:
import numpy as np
import pandas as pd


from keras.models import Sequential, Model, load_model
from keras.layers.core import Dense, Activation, Flatten
from keras.layers.embeddings import Embedding
from keras.layers.recurrent import LSTM
from keras.layers import Conv3D, Conv3DTranspose, Conv2D, Conv1D, MaxPooling1D, MaxPooling2D, Input, BatchNormalization, Dropout, Lambda, Bidirectional
from keras.layers import CuDNNLSTM, Reshape, PReLU, ZeroPadding2D, average
from keras.layers.merge import concatenate
from keras.optimizers import RMSprop, Adam, SGD
from keras.models import load_model
from keras.callbacks import Callback
from keras.utils import plot_model

from keras.preprocessing import image
import keras.backend as K

import matplotlib.pyplot as plt

import time 
import tensorflow as tf
from sklearn import metrics
import pickle
import os
import scipy.io as sio

import keras

import sklearn

Using TensorFlow backend.


In [0]:
import PIL

In [0]:
#from google.colab import files
#files.download("embeddings.npy")

In [0]:

np.random.seed(5)
tf.set_random_seed(5)

In [0]:

from keras.applications.mobilenet_v2 import MobileNetV2, preprocess_input

In [0]:
import h5py

In [0]:
def precision(y_true, y_pred):
    """Precision metric.
    Only computes a batch-wise average of precision.
    Computes the precision, a metric for multi-label classification of
    how many selected items are relevant.
    """
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    precision = true_positives / (predicted_positives + K.epsilon())
    return precision

In [17]:
len(os.listdir("/content/drive/My Drive/hdf5"))

101

In [0]:
class DataGeneratorJoint(keras.utils.Sequence):
    'Generates data for Keras'
    def __init__(self, filepath, list_batches, batch_size=1, shuffle=True):
        'Initialization'
        self.filepath = filepath
        self.list_batches = list_batches
        self.batch_size = batch_size
        self.shuffle = shuffle
        self.on_epoch_end()
        

    def __len__(self):
        'Denotes the number of batches per epoch'
        return len(self.list_batches)

    def __getitem__(self, index):
        'Generate one batch of data'
        # Generate indexes of the batch
        indexes = self.indexes[index*self.batch_size:(index+1)*self.batch_size]

        # Find list of IDs
        list_batches_temp = [self.list_batches[k] for k in indexes]
        
        # Generate data
        images, voxels = self.__data_generation(list_batches_temp, indexes)

        return [voxels, images], voxels
    
    def on_epoch_end(self):
        'Updates indexes after each epoch'
        self.indexes = np.arange(len(self.list_batches))
        if self.shuffle == True:
            np.random.shuffle(self.indexes)
            
            
    def __data_generation(self, list_batches_temp, indexes):
        'Generates data containing batch_size samples' 
        # Initialization
        with h5py.File(self.filepath+"/"+list_batches_temp[0]) as f:
            images = np.swapaxes(f['data'].value, 1, 3)
            voxels = np.swapaxes(f["label-voxel"].value, 1, 4)

        max_min_mean = ( images.max() + images.min() ) / 2
        images = (images - max_min_mean) / (images.max() - max_min_mean)  # scaling image pixels to range  -1 to 1

        images, voxels = sklearn.utils.shuffle(images, voxels) # shuffle within the batch itself
        
        return images[:, 1:225, 1:225, :], voxels

In [19]:
listdir = os.listdir("/content/drive/My Drive/hdf5")

import re
def sorted_aphanumeric(data):
    convert = lambda text: int(text) if text.isdigit() else text.lower()
    alphanum_key = lambda key: [ convert(c) for c in re.split('([0-9]+)', key) ] 
    return sorted(data, key=alphanum_key)
  
listdir_sorted = sorted_aphanumeric(listdir)
listdir_sorted

['batch_0.h5',
 'batch_1.h5',
 'batch_2.h5',
 'batch_3.h5',
 'batch_4.h5',
 'batch_5.h5',
 'batch_6.h5',
 'batch_7.h5',
 'batch_8.h5',
 'batch_9.h5',
 'batch_10.h5',
 'batch_11.h5',
 'batch_12.h5',
 'batch_13.h5',
 'batch_14.h5',
 'batch_15.h5',
 'batch_16.h5',
 'batch_17.h5',
 'batch_18.h5',
 'batch_19.h5',
 'batch_20.h5',
 'batch_21.h5',
 'batch_22.h5',
 'batch_23.h5',
 'batch_24.h5',
 'batch_25.h5',
 'batch_26.h5',
 'batch_27.h5',
 'batch_28.h5',
 'batch_29.h5',
 'batch_30.h5',
 'batch_31.h5',
 'batch_32.h5',
 'batch_33.h5',
 'batch_34.h5',
 'batch_35.h5',
 'batch_36.h5',
 'batch_37.h5',
 'batch_38.h5',
 'batch_39.h5',
 'batch_40.h5',
 'batch_41.h5',
 'batch_42.h5',
 'batch_43.h5',
 'batch_44.h5',
 'batch_45.h5',
 'batch_46.h5',
 'batch_47.h5',
 'batch_48.h5',
 'batch_49.h5',
 'batch_50.h5',
 'batch_51.h5',
 'batch_52.h5',
 'batch_53.h5',
 'batch_54.h5',
 'batch_55.h5',
 'batch_56.h5',
 'batch_57.h5',
 'batch_58.h5',
 'batch_59.h5',
 'batch_60.h5',
 'batch_61.h5',
 'batch_62.h5',
 '

In [0]:
training_generator = DataGeneratorJoint(filepath = "/content/drive/My Drive/hdf5",  list_batches = listdir_sorted)

In [0]:
# train from scratch
def joint_model_shared():

    """Joint model with shared embedding layer between autoencoder and convnet. Train from scratch."""

    input_vox = Input(shape=(20,20,20,1,), name = "input_ac")
    conv1 = Conv3D(96, 7, name = "conv1_ac")(input_vox)
    act1 = PReLU(name = "p_re_lu_1")(conv1)
    conv2 = Conv3D(256, 5, name = "conv2_ac")(act1)
    act2 = PReLU(name = "p_re_lu_2")(conv2)
    conv3 = Conv3D(384, 3, name = "conv3_ac")(act2)
    act3 = PReLU(name = "p_re_lu_3")(conv3)
    conv4 = Conv3D(256, 3, name = "conv4_ac")(act3)
    act4 = PReLU(name = "p_re_lu_4")(conv4)
    flat = Flatten(name = "flat_ac")(act4)
    embedding_ac = Dense(64, name = "dense1_ac")(flat)


    mobilenet_v2 = MobileNetV2(include_top = False, input_shape = (224,224,3))
    flat_mobilenet = Flatten(name = "flat")(mobilenet_v2.output)
    embedding_mn = Dense(64, name = "dense_out")(flat_mobilenet)

    embedding = Dense(64, name = "embedding") # Embedding layer

    shared_embedding_ac = embedding(embedding_ac)
    shared_embedding_mn = embedding(embedding_mn)
    merged_embedding = average([shared_embedding_ac, shared_embedding_mn])

    dens2 = Dense(216, name = "dense2_ac")(merged_embedding)
    reshp = Reshape((6, 6, 6, -1), name = "reshape_ac")(dens2)
    deconv1 = Conv3DTranspose(256, 3, name = "deconv1_ac")(reshp)
    dact1 = PReLU(name = "p_re_lu_5")(deconv1)
    deconv2 = Conv3DTranspose(384, 3, name = "deconv2_ac")(dact1)
    dact2 = PReLU(name = "p_re_lu_6")(deconv2)
    deconv3 = Conv3DTranspose(256, 5, name = "deconv3_ac")(dact2)
    dact3 = PReLU(name = "p_re_lu_7")(deconv3)
    deconv4 = Conv3DTranspose(96, 7, name = "deconv4_ac")(dact3)
    dact4 = PReLU(name = "p_re_lu_8")(deconv4)
    out = Conv3DTranspose(1, 1, activation = "sigmoid", name = "output_ac")(dact4)

    joint_model = Model(inputs = [input_vox, mobilenet_v2.input], outputs = out)
    
    # set lower conv layers in convnet to nontrainable
    print("Setting non-trainable layers...")
    for layer in joint_model.layers[:-46]:
        print(layer.name)
        layer.trainable = False

    return joint_model

In [0]:
# loading pretrained models
def joint_model_shared():

    """Joint model with shared embedding layer between autoencoder and convnet. Train from scratch."""

    input_vox = Input(shape=(20,20,20,1,), name = "input_ac")
    conv1 = Conv3D(96, 7, name = "conv1_ac")(input_vox)
    act1 = PReLU(name = "p_re_lu_1")(conv1)
    conv2 = Conv3D(256, 5, name = "conv2_ac")(act1)
    act2 = PReLU(name = "p_re_lu_2")(conv2)
    conv3 = Conv3D(384, 3, name = "conv3_ac")(act2)
    act3 = PReLU(name = "p_re_lu_3")(conv3)
    conv4 = Conv3D(256, 3, name = "conv4_ac")(act3)
    act4 = PReLU(name = "p_re_lu_4")(conv4)
    flat = Flatten(name = "flat_ac")(act4)
    embedding_ac = Dense(64, name = "dense1_ac")(flat)


    mobilenet_v2 = MobileNetV2(include_top = False, input_shape = (224,224,3))
    flat_mobilenet = Flatten(name = "flat")(mobilenet_v2.output)
    embedding_mn = Dense(64, name = "dense_out")(flat_mobilenet)

    embedding = Dense(64, name = "embedding") # Embedding layer

    shared_embedding_ac = embedding(embedding_ac)
    shared_embedding_mn = embedding(embedding_mn)
    merged_embedding = average([shared_embedding_ac, shared_embedding_mn])

    dens2 = Dense(216, name = "dense2_ac")(merged_embedding)
    reshp = Reshape((6, 6, 6, -1), name = "reshape_ac")(dens2)
    deconv1 = Conv3DTranspose(256, 3, name = "deconv1_ac")(reshp)
    dact1 = PReLU(name = "p_re_lu_5")(deconv1)
    deconv2 = Conv3DTranspose(384, 3, name = "deconv2_ac")(dact1)
    dact2 = PReLU(name = "p_re_lu_6")(deconv2)
    deconv3 = Conv3DTranspose(256, 5, name = "deconv3_ac")(dact2)
    dact3 = PReLU(name = "p_re_lu_7")(deconv3)
    deconv4 = Conv3DTranspose(96, 7, name = "deconv4_ac")(dact3)
    dact4 = PReLU(name = "p_re_lu_8")(deconv4)
    out = Conv3DTranspose(1, 1, activation = "sigmoid", name = "output_ac")(dact4)

    joint_model = Model(inputs = [input_vox, mobilenet_v2.input], outputs = out)
    
    # set lower conv layers in convnet to nontrainable
    print("Setting non-trainable layers...")
    for layer in joint_model.layers[:-46]:
        print(layer.name)
        layer.trainable = False
        
    autoenc = load_model("autoencoder8.h5", custom_objects = {"precision":precision})
    mobile_net = load_model("mobile_regress_5.h5", custom_objects = {"euclidean_distance_loss":euclidean_distance_loss})
        
    print("AUtoencoder")
    for layer in joint_model.layers:
      try:
        layer.set_weights(autoenc.get_layer(layer.name).get_weights())
      except ValueError:
         print("AC nema layera ", layer.name) 

    print("Mobilenet")
    for layer in joint_model.layers:
      try:
        layer.set_weights(mobile_net.get_layer(layer.name).get_weights())
      except ValueError:
         print("MN nema layera ", layer.name) 

    return joint_model

In [0]:
 # loss func
def euclidean_distance_loss(y_true, y_pred): 
    """ Euclidean distance loss https://en.wikipedia.org/wiki/Euclidean_distance 
    :param y_true: TensorFlow/Theano tensor 
    :param y_pred: TensorFlow/Theano tensor of the same shape as y_true :return: float """ 
    return K.sqrt(K.sum(K.square(y_pred - y_true), axis=-1))

In [24]:
joint_model = joint_model_shared()
joint_model.summary()

Setting non-trainable layers...
input_2
Conv1_pad
Conv1
bn_Conv1
Conv1_relu
expanded_conv_depthwise
expanded_conv_depthwise_BN
expanded_conv_depthwise_relu
expanded_conv_project
expanded_conv_project_BN
block_1_expand
block_1_expand_BN
block_1_expand_relu
block_1_pad
block_1_depthwise
block_1_depthwise_BN
block_1_depthwise_relu
block_1_project
block_1_project_BN
block_2_expand
block_2_expand_BN
block_2_expand_relu
block_2_depthwise
block_2_depthwise_BN
block_2_depthwise_relu
block_2_project
block_2_project_BN
block_2_add
block_3_expand
block_3_expand_BN
block_3_expand_relu
block_3_pad
block_3_depthwise
block_3_depthwise_BN
block_3_depthwise_relu
block_3_project
block_3_project_BN
block_4_expand
block_4_expand_BN
block_4_expand_relu
block_4_depthwise
block_4_depthwise_BN
block_4_depthwise_relu
block_4_project
block_4_project_BN
block_4_add
block_5_expand
block_5_expand_BN
block_5_expand_relu
block_5_depthwise
block_5_depthwise_BN
block_5_depthwise_relu
block_5_project
block_5_project_BN

In [0]:
joint_model.compile(optimizer = SGD(lr = 10**-6), loss = "binary_crossentropy", metrics = [precision])

In [26]:
!nvidia-smi

Thu Jan 31 13:40:39 2019       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 396.44                 Driver Version: 396.44                    |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  Tesla K80           Off  | 00000000:00:04.0 Off |                    0 |
| N/A   46C    P0    59W / 149W |   2173MiB / 11441MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
                                                                               
+-----------------------------------------------------------------------------+
| Processes:                                                       GPU Memory |
|  GPU  

In [27]:
history = joint_model.fit_generator(generator = training_generator, epochs = 10)

Epoch 1/10
 13/101 [==>...........................] - ETA: 1:51:30 - loss: 1.7538 - precision: 0.2681

ResourceExhaustedError: ignored

In [0]:

plt.plot(history.history["loss"], label = "loss")
plt.plot(history.history["precision"], label = "precision")
plt.plot(history.history["mean_absolute_error"], label = "mean_absolute_error")

plt.legend()



In [0]:

plt.plot(history.history["loss"], label = "loss")

In [0]:
joint_model.save("joint_model_3.h5")

In [0]:
!cat /proc/meminfo

MemTotal:       13335276 kB
MemFree:         6600856 kB
MemAvailable:   10989504 kB
Buffers:           69600 kB
Cached:          4754096 kB
SwapCached:            0 kB
Active:          1072252 kB
Inactive:        5234244 kB
Active(anon):     693332 kB
Inactive(anon):  1058152 kB
Active(file):     378920 kB
Inactive(file):  4176092 kB
Unevictable:           0 kB
Mlocked:               0 kB
SwapTotal:             0 kB
SwapFree:              0 kB
Dirty:             34160 kB
Writeback:             0 kB
AnonPages:       1482744 kB
Mapped:          1481316 kB
Shmem:           1058676 kB
Slab:             196976 kB
SReclaimable:     149552 kB
SUnreclaim:        47424 kB
KernelStack:        4656 kB
PageTables:        10544 kB
NFS_Unstable:          0 kB
Bounce:                0 kB
WritebackTmp:          0 kB
CommitLimit:     6667636 kB
Committed_AS:    4603752 kB
VmallocTotal:   34359738367 kB
VmallocUsed:           0 kB
VmallocChunk:          0 kB
AnonHugePages:         0 kB
ShmemHugePages:  

In [0]:
!cat /proc/cpuinfo

processor	: 0
vendor_id	: GenuineIntel
cpu family	: 6
model		: 79
model name	: Intel(R) Xeon(R) CPU @ 2.20GHz
stepping	: 0
microcode	: 0x1
cpu MHz		: 2200.000
cache size	: 56320 KB
physical id	: 0
siblings	: 2
core id		: 0
cpu cores	: 1
apicid		: 0
initial apicid	: 0
fpu		: yes
fpu_exception	: yes
cpuid level	: 13
wp		: yes
flags		: fpu vme de pse tsc msr pae mce cx8 apic sep mtrr pge mca cmov pat pse36 clflush mmx fxsr sse sse2 ss ht syscall nx pdpe1gb rdtscp lm constant_tsc rep_good nopl xtopology nonstop_tsc cpuid tsc_known_freq pni pclmulqdq ssse3 fma cx16 pcid sse4_1 sse4_2 x2apic movbe popcnt aes xsave avx f16c rdrand hypervisor lahf_lm abm 3dnowprefetch invpcid_single pti ssbd ibrs ibpb stibp fsgsbase tsc_adjust bmi1 hle avx2 smep bmi2 erms invpcid rtm rdseed adx smap xsaveopt arat arch_capabilities
bugs		: cpu_meltdown spectre_v1 spectre_v2 spec_store_bypass l1tf
bogomips	: 4400.00
clflush size	: 64
cache_alignment	: 64
address sizes	: 46 bits physical, 48 bits virtual
power ma

In [0]:
!kill

kill: usage: kill [-s sigspec | -n signum | -sigspec] pid | jobspec ... or kill -l [sigspec]


In [0]:
!kill -9 -1